In [31]:
import random
import string
from password_strength import PasswordStats
import numpy as np

def random_string (N):
  return ''.join(random.choices(string.ascii_letters+string.digits, k=N))
          
all_strgs = []    
    
fg = open('goodpasswords.txt','w')
fb = open('badpasswords.txt','w')
count_good = 0
count_bad = 0
while(count_good<2000 or count_bad<2000):
   psw = random_string(15) 
   stats = PasswordStats(psw)
   strg = (stats.strength())
   all_strgs.append(strg)
   if strg > 0.69 and count_good<2000:
      count_good += 1
      fg.write(psw+','+str(strg)+'\n')
      #print ('good')
      #print (psw+','+str(strg))
   elif strg < 0.69 and count_bad<2000:
      count_bad += 1
      fb.write(psw+','+str(strg)+'\n')
      #print ('bad')
      #print (psw+','+str(strg))
      
fb.close()
fg.close()

0.6944740189330518

In [26]:
def all_subwords(word):
  # get all unique subwords of the words and present them in a sorted way
  subword_list = []
  for j in range(len(word)):
    for i in range(len(word)-j):
       if word[i:i+j+1] not in subword_list:
         subword_list.append(word[i:i+j+1])
  return sorted(subword_list) 

def m_matrix(lmbda, mu):
  # lambda and mu are subwords of word
  M = {}
  for i in range(len(lmbda)):
    for j in range(len(mu)):
      if lmbda[i] == mu[j]:
         M[(i,j)] = mu[j]
      else:
         M[(i,j)] = '*'
  return M   

def Gamma (M):
  # constructing graph corresponding to two subword matrix M
  gamma = {}
  for el in M:
    gamma[el] = []
    if (el[0]+1, el[1]+1) in M:
       if M[el] != '*' and M[(el[0]+1, el[1]+1)] != '*':
                gamma[el].append((el[0]+1, el[1]+1))

  return gamma
def DFS(temp, v, gamma, visited):
    visited[v] = True
    temp.append(v)
    for i in gamma[v]:
        if visited[i] == False:
                temp = DFS(temp, i, gamma, visited)
    return temp

def connectedComponents(gamma):
        visited = {}
        cc = []
        for el in gamma:
           visited[el] = False
        for v in gamma:
            if not visited[v]:
                temp = []
                cc1 = DFS(temp, v, gamma, visited)
                cc.append(cc1)      
        return cc     

def wc(cc, M):
  #list of words from connected components
  wcs = {}
  for c in cc:
      w = ''
      for i in c:
         w += M[i]
      if w in wcs:
         wcs[w] +=1
      else:
         wcs[w] = 1  
  return wcs      


In [32]:
import numpy as np
import random
import string
import math

from numpy import asarray
from numpy import savez_compressed

names = []

for line in open('goodpasswords.txt'):
    word =line.strip().split(',')[0]
    names.append(word)
    
for line in open('badpasswords.txt'):
    word =line.strip().split(',')[0]
    names.append(word)

print (len(names))
    
counter = 0
for word in names:
  print((counter,word))


  subwords = all_subwords(word)
  dim = 210 
  
  pairs = [(a, b) for idx, a in enumerate(subwords) for b in subwords[idx:]]
  m_matrices = [m_matrix(el[0],el[1]) for el in pairs]
  gammas = [Gamma(m) for m in m_matrices]
  ccs = [connectedComponents(gamma) for gamma in gammas]
  wcss = [wc(ccs[i], m_matrices[i]) for i in range(len(ccs))]


  mtensor = np.zeros((dim,dim,dim))
  for i, el in enumerate(subwords):
      mtensor[i+1,0,0] = len(el)
      mtensor[0,i+1,0] = len(el)
  mtensor[0,0,0] = 1
  for i in range(len(subwords)):
     for j in range(len(subwords)):
         if (subwords[i], subwords[j]) in pairs: 
             position = pairs.index((subwords[i], subwords[j]))
             mc = wcss[position]
             for el in mc:
                if el == '*':
                  mtensor[i+1,j+1,0] = mc[el]
                else:
                  wpos = subwords.index(el)
                  mtensor[i+1, j+1, wpos+1] = mc[el]  
         if (subwords[j], subwords[i]) in pairs:
             position = pairs.index((subwords[j], subwords[i])) 
             mc = wcss[position]
             for el in mc:
                if el == '*':
                  mtensor[j+1,i+1,0] = mc[el]
                else:
                  wpos = subwords.index(el)
                  mtensor[j+1, i+1, wpos+1] = mc[el]   
  

  data = asarray(mtensor)
  savez_compressed('psw'+word+'.npz', data)
  counter += 1
 



4000
(0, 'rNuo38K5volHQFy')
(1, 'zdDsmveglRJON1m')
(2, 'ipH7gaQ3vbmvAef')
(3, 'UqrNKOyfoZ31xfe')
(4, 'SFI2hCQRGUIgrBL')
(5, 'F59oTbtXeFxCQsY')
(6, 'G2zFVEICToXdyjD')
(7, 'JEo3IOig2LC0GjN')
(8, 'QOlFf6LMZBHIUmY')
(9, '492R1ZAhlJk6sMl')
(10, 'rmCYGNxjSoteZBj')
(11, 'zGTFLZ4IsAfSXA0')
(12, 'r5hRX6GijqysTC2')
(13, 'ZVPXcIxZmkGdE5e')
(14, 'kTLYN8tdlWhsOhp')
(15, '8ojIUE9Avwfypcn')
(16, 'MxYFOQQlgjJwV1T')
(17, 'iaB9jcukn5FszTL')
(18, 'ZokesxUh6qEINh3')
(19, '9AnrdkoUMsLwJXz')
(20, 'maZfAYdpvhtag4I')
(21, 'qz3EuxVf9XRLrnE')
(22, 'uaMAIpcDqeMPV8t')
(23, 'qvrWDGS904CUuYI')
(24, 'n7vQUqk39lje9KG')
(25, 'aQZVDqjVvNp5MxF')
(26, 'IAQmVspvyVMh9oL')
(27, '9qRMoz0iXJyHm13')
(28, 'QxjFbimR5C0UdT8')
(29, 'k7l5vrslHXCoL83')
(30, 'KggfpYNhFmBnv5d')
(31, 'FzPZajWXvyYob4t')
(32, 'ocnFjRVlsgD9aDB')
(33, 'OGkPTW2iuwJVn3O')
(34, 'c1jaSCGggXQlYR4')
(35, 'm79zFdryVnGCTSN')
(36, 'uJZLNbs4CDzgquQ')
(37, '5iaEFNwFyIYqsWD')
(38, 'Om2GqSWHg7PuyCt')
(39, 'zTlx93m71X8wJG3')
(40, 'HFABauy5zPbipXe')
(41, '82oEiClAtneqJuZ

(332, 'XR2PJ53LtxDjAYn')
(333, 'SpyiGpJxElQeACN')
(334, '9q3JYkKu8acdP3w')
(335, 'ZOdKXE0MctBUc8g')
(336, 'D0V4ysrJWl5f8Eg')
(337, '4Xr9RepPWJs7ibS')
(338, 'qfGMRq408KadC7g')
(339, 'UMreNxBH08IGrO4')
(340, 'FNa0CTLv7KreuhS')
(341, 'YuHhvKnSAQHqoNc')
(342, 'ATOsKBdXtMEU5iC')
(343, 'EcGgRtePguNazdw')
(344, '6U5XXIMT1DhxvYl')
(345, 'XarZdHS4wUmxA3y')
(346, '2kQg8aKuOE6t1f3')
(347, '1pqW4uALBtk2sRr')
(348, 'jqRmZ1rHf4VsvL6')
(349, '4SOC2NoBmYadHL3')
(350, 'QMkdWvns0C27HNW')
(351, 'aJ2JkiRM4bAHDdo')
(352, 'NU1LPG5CEjkxS4B')
(353, 'nPw6YAbSsnvBhZL')
(354, 'iXsmwMJxjNpK3Gg')
(355, 'fP6mEJTbFWZE3DQ')
(356, 'jtDHGsrpGScCVF4')
(357, 'DPd9Sq7z34Ndjiw')
(358, 'BuXCbYI4mULdaDk')
(359, 'nw0tRlIuoPRvbxA')
(360, 'Tj2q1iUtReco8ch')
(361, 'VDvExR6MN3tOmSS')
(362, '0eWmTd20JOYDrGv')
(363, 'ZilRy9jFjGWJm0H')
(364, 't7nBmzPvGerwyZA')
(365, 'x7CFTzohTlUQsrH')
(366, 'fmZvipmo0MWGuUq')
(367, 'UImMDSNxVaLZPZJ')
(368, 'RKEuCQakFD9OpBp')
(369, 'anXDey2DRiL5BsG')
(370, '6oZe7LFPwXTjSWc')
(371, 'jVb9EyQFOhEWZ5S')


(660, '4rnyKrUq1ILXbOH')
(661, '7NQrnRL9VMa47EK')
(662, 'u41CzdhLEwFUABE')
(663, 'oD9A6aoUBKgYFxh')
(664, 'qahVCPQeoWQL6RN')
(665, 'vtX0l5SbKCCAVFy')
(666, '7oRXZ7l1ADIe0sv')
(667, 'EoMyV7oRuFUb5nk')
(668, 'ctvoXLC9Y80YIbq')
(669, 'ImifaQpWJoVItrN')
(670, 'MZJAm32vcqztoSY')
(671, 't6z1CxIhtpsJ8S2')
(672, 'jcbKGeI3Qu3VdCq')
(673, 'jlwFZmB37aSMf7e')
(674, '1Bdm8Tgnb27cve2')
(675, 'tzw0QAMBYGPb6we')
(676, 'ocpNfRwsxT7MltT')
(677, '8ipQKQR0fbBLdA9')
(678, 'YpoeFvkGWKsZ3Nb')
(679, '8uXaVTsEz57ey7l')
(680, 'JpAmGlozuXv4WfX')
(681, 'f2JtpraKudSF8zW')
(682, 'BpQa4Rlw9TiNSBD')
(683, 'pK4ebXvicYzVMAB')
(684, 'F5ojGaiGeVXTYsm')
(685, '1Vgx92lBe4UBOb3')
(686, '1Na1JK9vwhgd7qy')
(687, 'KHWGVYMMpcFsSqx')
(688, 'nuNsR5endarVDQ1')
(689, 'fjJMv82dyVpGAWm')
(690, 'DfJA1kMs3g4WXfx')
(691, '76H1WGuchYedk8A')
(692, '2votzyg7BlaSfo8')
(693, 'ICnXWOv1qs61mar')
(694, 'EuPMAdLBv9IVVzW')
(695, 'cIrf2Z9qbGFi8o3')
(696, '5h61BGCTLMHEP7Y')
(697, '4O1kVn9q2hByJKS')
(698, 'qafCID1KYYMkwF3')
(699, 'V5JHpcnzdXxq7jK')


(988, 'MwwzPnRKW7ODeJt')
(989, 'FcDOFUXkQLE7Tth')
(990, '7aPedkjtgKRwSrn')
(991, 'HemMDOTVXr0x4Oh')
(992, 'EEwkZG9bD5mBNHO')
(993, 'aCXKjy3AeYrqNa2')
(994, '3yzt2AXNOWhS5x5')
(995, 'aZV4WAP8tQiq6H0')
(996, 'Q7BviG85ed6G0xc')
(997, 'CLmxBtc93VLayKT')
(998, 'qgRyIs3A9WbULWl')
(999, 'GJ87uVWBMnRWCPd')
(1000, 'vuKjlALMmaDfeKI')
(1001, '9m4zxw5ZVcFbm0S')
(1002, 'zwliPAvjZxBoHnT')
(1003, 'M4D4sLlhOxyTp3F')
(1004, 'VEX2gH48wcDer5D')
(1005, 'R9uJHkeAoBlYSiJ')
(1006, 't7JAk5Tc4ADqwLp')
(1007, '6mbNEXSf1RWN4C3')
(1008, '68jHjeIAoBfFuq2')
(1009, 'uS8E6PaJdUkhYa2')
(1010, 'sqWmHi95T1mxeg6')
(1011, 'xXtjGVCo6ItSWAg')
(1012, '9mmyc53617V4ILF')
(1013, 'w2uMldRfPSyKx46')
(1014, 'cY06Au1mfsn0eQl')
(1015, '9QWOKn4bgwPj5OE')
(1016, 'BnFWAGlfcOPTcJ9')
(1017, 'pUlIEhNsX6anab3')
(1018, 'uF6aRcESvy7NmeZ')
(1019, 'LoevQqJxiVX4AUR')
(1020, 'YpeUaLq7fbA32CF')
(1021, 'rBvd9CF6Jj0tpAR')
(1022, 'FQCd9hl5WXPGclt')
(1023, '3ojdmUAAEB8Twn1')
(1024, 'eSou7mG8xlL2N5r')
(1025, 'VkvYzKlPfx3c0jJ')
(1026, 'OH6o6b3gaNf1AEk'

(1304, '6T7kuBIa3GXIFfx')
(1305, 'bIAi2xKCgOvk9mK')
(1306, '0aoNjRUvW58lnBg')
(1307, 'TOQ7ydz8VMuvQDE')
(1308, '8ciRrdXuoUa5Wpc')
(1309, 'tVGnkRjThRmwrlN')
(1310, 'COY5RUJbDAlSpBf')
(1311, 'HxAvmWlX1dzh9qG')
(1312, 'zsvTndahaNy4DP3')
(1313, 'PIvFH13CX5M7F2Y')
(1314, 'NPEf3ihb6hymrjk')
(1315, 'O193sCwumel73cW')
(1316, 'ftvCcRyPGDgnWG9')
(1317, 'bwdBqYnvCmXy052')
(1318, 'klnp6jsWHKBNVtL')
(1319, 'nnFkAVKLgJGTUMR')
(1320, 'qn23vxfpo0T1Xx6')
(1321, 'iWcugzRksVUmQZd')
(1322, '2ANlMzZFQraoWuN')
(1323, '1Z5KQALzUqec9wO')
(1324, 'Ix4RCi7B6rlgUgo')
(1325, 'bprZ6HFmWcu2dxM')
(1326, '5wgJ6aoefPfBKyb')
(1327, '3UWmzycbsag0sFe')
(1328, 'uBl4N0vHzEe7Tp6')
(1329, 'LZv4aqHtGW64ml7')
(1330, 'XbaMYzXBpR7joPH')
(1331, 'ijeTakGam2yULQp')
(1332, 'PZAMG3IF4zVbZNu')
(1333, 'AMW7ujSDh4P6pv9')
(1334, 'go46OHyTElQrzJL')
(1335, '1Nf01s7wQHLVYUZ')
(1336, '7Ik3MNXHWYAty4I')
(1337, 'YwBPacVdFxUkSsi')
(1338, 'a0IinOsPtz35qgE')
(1339, 'tKkcolyNdSRH92F')
(1340, 'NSCnvu1c7QX5hQ6')
(1341, '5ImIOHTZSD1Kzcw')
(1342, 'Y2x1

...

(2568, 'pYmFFQT2gdyHTT4')
(2569, '0804wRd7ZaaMwCI')
(2570, 'uFBeu2R5UQDFUOm')
(2571, 'vE4ExxzVT0e4pBl')
(2572, 'PaRuCdd11LgonYH')
(2573, 'JmuBkVkBjiAo05t')
(2574, 'LySZjZjVx1tNkhF')
(2575, 'Tkn7WNo8SyD7w8j')
(2576, 'xlVNRjzgpXOVKig')
(2577, '34R8XVDcAFYo8R9')
(2578, '9zUGbEtIzy79NtP')
(2579, 'WGs6efsjaL1lilg')
(2580, 'ttcByjsrrcvapkm')
(2581, 'ODiHzsoCw53DC1L')
(2582, 'EElW7BlLkEQ7LST')
(2583, 'M45m5Tc9o8xP9Ex')
(2584, 'zAOo0AqmG0TmmwA')
(2585, 'pPkxlCLxpCtnXL0')
(2586, 'WUa0dWdG73jIlcz')
(2587, 'a7EyhnvBhkJmFvn')
(2588, 'E1rn9Xa1CNljSXa')
(2589, '99WqcEjCpSOjWmk')
(2590, 'bMOzxXFzxMMv0rT')
(2591, 'o4x0GiU34mfFhx0')
(2592, 'LwgxaPccD8ajMze')
(2593, '2xG2VwTstN4JBGE')
(2594, 'MebL0oO9b0fz29B')
(2595, 'ttSI4yZdc1rBABT')
(2596, 'UKhUtUXytvJNEeb')
(2597, 'nnxLgMnDuhogoqd')
(2598, 'dgM9LYq10gSqo7u')
(2599, 'ck2IL1k3DXXTw4O')
(2600, 'uBaisgaT2tBqq1q')
(2601, 'Fpog7CYwYnK4ofd')
(2602, 'Q7N47BvKyoQULTB')
(2603, 'lCooh4Vn7QxlCxo')
(2604, 'RfjbTabw75a70qg')
(2605, 'bcpFLF9C62MQfgf')
(2606, '6LLi

(2884, 'XXjiXobANtON1D2')
(2885, 'ss3BR0R0L0Hb7xA')
(2886, 'AtrL6cl6r7KEvac')
(2887, 'hMtLi635ssROsHA')
(2888, 'yuh3yymCNUluN3f')
(2889, 'Mf2goyy7yCDSNoC')
(2890, 'b3lVsh3M4bH5seU')
(2891, 'QW8z0hHuM0wftut')
(2892, 'M3zdPvYnCzhL1kd')
(2893, 'MEpqBacjcMI9Opj')
(2894, 'EA21yGMNd80EdcP')
(2895, 'Bi1dg247mU6RBBh')
(2896, 'D7nYPhcUZA2mAhB')
(2897, '3vZYL0QG18XH1jL')
(2898, 'FMbOXVgrJMCypp9')
(2899, 'V0rDpzeYSz6So4L')
(2900, 'xUUW9sM2wMeVIwe')
(2901, '0LVJAg61v7Lmgv0')
(2902, 'lNwSwrdXrFJoV1f')
(2903, 'tFUAuEA9aK63P6Y')
(2904, 'xz6LYjDY4kjKAeZ')
(2905, 'Qhf8oPPlzsJREpl')
(2906, 'Ul3jcAiTrWj6nl4')
(2907, 'Kwdxg8swRKB8xgP')
(2908, 'goPKK22KnJH5dzM')
(2909, 'eJIIbVuJxpeygRb')
(2910, 'RwGlAw828gVc0Fl')
(2911, '3wx0Wkcw65uqqnX')
(2912, '6Ut3g47QQx3BmPL')
(2913, '0YiPcw2WWcYFxjs')
(2914, 'j42Da84u8uyM8hw')
(2915, 'NSMhgUG848Qvh2E')
(2916, '7KlV1hJOHXVJWD5')
(2917, 'HElEBmBtyWQuz1r')
(2918, 'pAVNWLXfKhhpgaG')
(2919, 'ZaypsHp5RlsufKF')
(2920, '8tIwDE7nL7eYxYm')
(2921, 'PmsaP9IpQsXV1d4')
(2922, 'hoGX

(3200, 'kvM2SOExD7AEdbM')
(3201, 'l3eX3y8Ou9AsjOR')
(3202, 'MRdcO5qOe7hqKZs')
(3203, 'WBltl22wSfnehVD')
(3204, 'ej7j7ZMGhWK3uKY')
(3205, 'lqZTPF78wUFA7q9')
(3206, '8618eqgHc5V96oS')
(3207, '2YRfVZZkxQ2u8b9')
(3208, 'bap6N65bcKEQWKM')
(3209, 'LL0wE7DxWKQKUUG')
(3210, 'xl8Zfak8kPnjynh')
(3211, '1yf0r1iguTPT8H1')
(3212, 'xFhIoiqCDVnnhxw')
(3213, 'GUBRygGegITddIX')
(3214, 'n0VA1nX5bgV3G6V')
(3215, 'LvSTl5XRMFTvqUu')
(3216, '8FbFQE51xbf2LWQ')
(3217, '2sXsbUdSLursgGq')
(3218, '08YIUUd2SbD8gEd')
(3219, 'ZsIYUvICV4NMHjj')
(3220, 'lN0OWC4CWImhur1')
(3221, 'vMvlPrDHDLhYNPC')
(3222, 'vwi7DZavvTAKWG2')
(3223, 'WT0wpD17WK7y5n0')
(3224, 'exCUZ1FWxbCHQPI')
(3225, 'isV5GCq7u06xj66')
(3226, 'sYrljTAS9VTsxD2')
(3227, 'YoahG1SU5jIxHoG')
(3228, 'NgAqJu9hirH6utq')
(3229, 'Xb63mm2FLulFtcn')
(3230, '3ZplZNv9KMijRM1')
(3231, 'pfpvs8Xn4uTmhht')
(3232, '9tW2AZiEUQVSzEV')
(3233, 'OFsri42oZohbsYS')
(3234, '7oDDd3SDQ87VKMu')
(3235, 'QEoH8V5EUaAjvvb')
(3236, 'b3qMxOyDqMHXjJ9')
(3237, 'zVoRAoSxd7WGqvW')
(3238, 'ISvU

(3516, 'vJQQlyz9FZ3sWrl')
(3517, 'BaaLvAZNf6Ay3Ve')
(3518, 'LOJqQTm1ONkqHAA')
(3519, 'jhmkoGIRmOKJ5KT')
(3520, 'cwDTa4TegMC6THO')
(3521, 'YWmZHBgcpTnYhhQ')
(3522, 'vQh9sseFBZlhscG')
(3523, 'FH3VGwlUUv4JH0K')
(3524, 'mKl8UqZFCULeKTG')
(3525, 'eJYnU4QLEEo0a4B')
(3526, 'GNjJxexRNcqPfr0')
(3527, 'ZKsbqrUey3Wb1Or')
(3528, '1dqnqSdot9nPdlU')
(3529, 'HeYY2IC8IznmMCy')
(3530, 'MF5sr0BCJMk5N4A')
(3531, 'kK15sYcyzCTl1ZK')
(3532, 'aTUu9sBHSeu6sDT')
(3533, '5rZYvpP2ExrgEqZ')
(3534, 'NNsv4obec1ZB15i')
(3535, '4N6BaqGwiZ4wG5Z')
(3536, 'UwNuumVcNiFcf96')
(3537, '5NRYSD7JBDn0a3R')
(3538, 'CWXUHYCmrllLajJ')
(3539, '5wWqBlej1z9lySS')
(3540, 'tp2uABdu7QQ5pVO')
(3541, 'i1oCFFR34Y39kuM')
(3542, 'XMQXX8xJSJyhdWh')
(3543, '6RIzBciyN2ShmNm')
(3544, 'vTUMRXb4I9lR566')
(3545, '5fmWrWulM4bB6rr')
(3546, 'n9irY9PNgMhpnMS')
(3547, 'PShu8Ou4vS7QbW0')
(3548, '9DxHZyBsNdB0v5D')
(3549, 'T6DF3w5Fu7wXYVm')
(3550, 'eWLx2NPxvukVwkK')
(3551, 'RfW3f23DTAKnfFC')
(3552, 'NIUlNjyvDQJiR1i')
(3553, 'iyasf1OIbb7dYh1')
(3554, '3x0t

(3832, 's2CbCwqXlu2099s')
(3833, 'wMaapaGaHEkXuv5')
(3834, 'HWJhI89WFMHGnKW')
(3835, 'KAKoKzieb4fnetL')
(3836, 'eqY0UqOiMeHRClI')
(3837, 'VYP9jHKHDdM9zBs')
(3838, 'F4QKjXjCVcgtuKt')
(3839, 'p6M5A27Qzm8UQ6d')
(3840, 'NbEbNSs6iCj1vBp')
(3841, 'dWhILDbMALZiZvW')
(3842, 'UYUjYBSf4lpCaGL')
(3843, 'mMA3zCz8m1fr9v9')
(3844, 'H6HImb8W06sYMbw')
(3845, 'hlSUj0qZSox9Zmp')
(3846, 'qQXqXmZppFBvfQJ')
(3847, '8WhJJduY0AOWWW1')
(3848, 'XLc1nUXtNpjS1FM')
(3849, 'B2UmynvFKulaKL2')
(3850, 'Vsgke9keH90mLq9')
(3851, 'dNn0MBEOpxrUES0')
(3852, 'ibgzWZYFWYc6W66')
(3853, 'PvfchQa7lybQl6B')
(3854, '92O4MFqgcqHr1FP')
(3855, 'QVH4rPFYHsQr7mz')
(3856, 'J37j71Sf1kOriAE')
(3857, 'QXyk0LQAHCxxocv')
(3858, 'MMcJvXGDpsUPYvU')
(3859, '6F9qCBkzqYU61Oj')
(3860, 'FYDZOhPaaDemjhB')
(3861, 'CD93Ahv8pFO3qpw')
(3862, 'Uzh09BbBkpATT5D')
(3863, 'AahkxLmWIbkGaF4')
(3864, 'nDPbvOdxhbv056K')
(3865, 'iWANfChd8HIRAXA')
(3866, 'AEELaRapXQWlG2w')
(3867, '7eToSIX3wWFFKkK')
(3868, 'ej9lAZ1s4st2jj8')
(3869, 'msf6ITxsHHw7RRa')
(3870, '6eWf